<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/debugg/debugging_apis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Se entrega a mas tardar el martes 14 de Noviembre a las 11:55 pm. Vale 40 puntos de tarea.
Te recomiendo leer todo el ejercicio antes de empezar a ahcer codigo, para que te des una idea de que formato y estructura necesitamos

In [51]:
import pandas as pd
import requests
import json
import pyarrow.parquet as pq

Objetivos del Ejercicio:

    Realizar una solicitud a la API de Pokémon para obtener datos sobre un Pokémon específico.
    Cargar y visualizar los datos en bruto para comprender su estructura.
    Limpiar y organizar los datos para un análisis posterior.
    Guardar los datos limpios en un archivo CSV.

El objetivo es crear un `DataFrame` (tidydata) de varios pokemones que ademas contengan metadata de sus moviemientos.  
Para ello deberas realizar request al `API` de pokemon, entender como vienen los datos y despues combinarlos con los `joins` correspondientes.

# API request

Crea codigo para acceder al API de pokemon y hacer requests

Compelta el siguiente snippet para obter los datos del API de pokemon.

## Pokemon Data

Para  el tipo de pokemon debes obtener:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  

Guarda los `moves` del pokemon pues los usaremos mas adelante. En especial el `name` y la `url` de cada movimiento apra que podamos iterar despues

In [2]:
# Elige tu pokemon
pokemon_name = 'lucario'

In [4]:
# Solicitud para obtener datos de un Pokémon
pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
pokemon_response = requests.get(pokemon_url)
pokemon_data = pokemon_response.json()
lucario_map = {
    'id': pokemon_data['id'],
    'name': pokemon_data['name'],
    'types': [type_info['type']['name'] for type_info in pokemon_data['types']],
    'moves': [{'name': move['move']['name'], 'url': move['move']['url']} for move in pokemon_data['moves']]
}
lucario_map

{'id': 448,
 'name': 'lucario',
 'types': ['fighting', 'steel'],
 'moves': [{'name': 'mega-punch', 'url': 'https://pokeapi.co/api/v2/move/5/'},
  {'name': 'ice-punch', 'url': 'https://pokeapi.co/api/v2/move/8/'},
  {'name': 'thunder-punch', 'url': 'https://pokeapi.co/api/v2/move/9/'},
  {'name': 'swords-dance', 'url': 'https://pokeapi.co/api/v2/move/14/'},
  {'name': 'mega-kick', 'url': 'https://pokeapi.co/api/v2/move/25/'},
  {'name': 'headbutt', 'url': 'https://pokeapi.co/api/v2/move/29/'},
  {'name': 'body-slam', 'url': 'https://pokeapi.co/api/v2/move/34/'},
  {'name': 'take-down', 'url': 'https://pokeapi.co/api/v2/move/36/'},
  {'name': 'roar', 'url': 'https://pokeapi.co/api/v2/move/46/'},
  {'name': 'hyper-beam', 'url': 'https://pokeapi.co/api/v2/move/63/'},
  {'name': 'low-kick', 'url': 'https://pokeapi.co/api/v2/move/67/'},
  {'name': 'counter', 'url': 'https://pokeapi.co/api/v2/move/68/'},
  {'name': 'strength', 'url': 'https://pokeapi.co/api/v2/move/70/'},
  {'name': 'earthqua

## Moves Data

Ahora para cada movimiento `move` del pokemon debes obtener los siguientes datos usando la `url`
1. `name`
2. `accuracy`
3. `type`
4. `power`
5. `pp`

In [16]:
# Solicitud para obtener datos de movimientos de un Pokémon

for move in moves: 
    # Realiza la solicitud a la API para obtener detalles del movimiento
    moves_response = requests.get(move['url'])
    move_data = moves_response.json()
    # Extrae los datos solicitados del movimiento
    move_map = {
        'name': move_data['name'],
        'accuracy': move_data['accuracy'],
        'type': move_data['type']['name'],
        'power': move_data['power'],
        'pp': move_data['pp']
    }
    print(move_map)

{'name': 'mega-punch', 'accuracy': 85, 'type': 'normal', 'power': 80, 'pp': 20}
{'name': 'fire-punch', 'accuracy': 100, 'type': 'fire', 'power': 75, 'pp': 15}
{'name': 'ice-punch', 'accuracy': 100, 'type': 'ice', 'power': 75, 'pp': 15}
{'name': 'thunder-punch', 'accuracy': 100, 'type': 'electric', 'power': 75, 'pp': 15}
{'name': 'mega-kick', 'accuracy': 75, 'type': 'normal', 'power': 120, 'pp': 5}
{'name': 'headbutt', 'accuracy': 100, 'type': 'normal', 'power': 70, 'pp': 15}
{'name': 'body-slam', 'accuracy': 100, 'type': 'normal', 'power': 85, 'pp': 15}
{'name': 'take-down', 'accuracy': 85, 'type': 'normal', 'power': 90, 'pp': 20}
{'name': 'double-edge', 'accuracy': 100, 'type': 'normal', 'power': 120, 'pp': 15}
{'name': 'disable', 'accuracy': 100, 'type': 'normal', 'power': None, 'pp': 20}
{'name': 'hyper-beam', 'accuracy': 90, 'type': 'normal', 'power': 150, 'pp': 5}
{'name': 'submission', 'accuracy': 80, 'type': 'fighting', 'power': 80, 'pp': 20}
{'name': 'counter', 'accuracy': 100,

# API Request para varios pokemons

Ahora que sabes como funciona el `API` realiza las peticiones de los siguientes pokemones respecto a sus caracteristicas
y movimientos, y guardalas en una lista, diciconario o tabla (lo que mas te acomode). 

In [19]:
pokemons = ['lucario', 'pikachu', 'gengar', 'charizard', 'blastoise', 'venusaur', 'dragonite', 'mew', 'mewtwo', 'garchomp']

In [26]:
# Solicitudes para obtener datos de un Pokémon
pokemon_data_dict = {}
movs_data_dict = {}
for pokemon in pokemons:
    # Datos del Pokémon
    pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon}'
    pokemon_response = requests.get(pokemon_url)
    pokemon_data = pokemon_response.json()
    movesp = [{'name': move['move']['name'], 'url': move['move']['url']} for move in pokemon_data['moves']]
    types = [type_info['type']['name'] for type_info in pokemon_data['types']]
    pokemon_info = {
        'id': pokemon_data['id'],
        'name': pokemon_data['name'],
        'type1': types[0] if len(types) > 0 else None,
        'type2': types[1] if len(types) > 1 else None,
        'moves': movesp
        }
    pokemon_data_dict[pokemon] = pokemon_info
for move in moves: 
    # Realiza la solicitud a la API para obtener detalles del movimiento
    moves_response = requests.get(move['url'])
    move_data = moves_response.json()
    # Extrae los datos solicitados del movimiento
    move_map = {
        'name': move_data['name'],
        'accuracy': move_data['accuracy'],
        'type': move_data['type']['name'],
        'power': move_data['power'],
        'pp': move_data['pp']
    }
    movs_data_dict[move['name']] = move_map
print(pokemon_data_dict)
print(movs_data_dict)

{'lucario': {'id': 448, 'name': 'lucario', 'type1': 'fighting', 'type2': 'steel', 'moves': [{'name': 'mega-punch', 'url': 'https://pokeapi.co/api/v2/move/5/'}, {'name': 'ice-punch', 'url': 'https://pokeapi.co/api/v2/move/8/'}, {'name': 'thunder-punch', 'url': 'https://pokeapi.co/api/v2/move/9/'}, {'name': 'swords-dance', 'url': 'https://pokeapi.co/api/v2/move/14/'}, {'name': 'mega-kick', 'url': 'https://pokeapi.co/api/v2/move/25/'}, {'name': 'headbutt', 'url': 'https://pokeapi.co/api/v2/move/29/'}, {'name': 'body-slam', 'url': 'https://pokeapi.co/api/v2/move/34/'}, {'name': 'take-down', 'url': 'https://pokeapi.co/api/v2/move/36/'}, {'name': 'roar', 'url': 'https://pokeapi.co/api/v2/move/46/'}, {'name': 'hyper-beam', 'url': 'https://pokeapi.co/api/v2/move/63/'}, {'name': 'low-kick', 'url': 'https://pokeapi.co/api/v2/move/67/'}, {'name': 'counter', 'url': 'https://pokeapi.co/api/v2/move/68/'}, {'name': 'strength', 'url': 'https://pokeapi.co/api/v2/move/70/'}, {'name': 'earthquake', 'url'

# Evolucion a  Tabular

Ahora para cada request (`pokemon` y `movimiento`) convierte su informacion a datos tabulares. Toma en cuenta que vamos a realizar operaciones sobre esos DataFrames, por lo tanto tienes que elegir un formato adecuado.  
Para cada pokemon, y para cada movimiento debe existir un `DataFrame`

In [29]:
# Crear un DataFrame para los Pokémon
pokemon_df = pd.DataFrame.from_dict(pokemon_data_dict, orient='index')

# Crear un DataFrame para los movimientos
moves_df = pd.DataFrame.from_dict(movs_data_dict, orient='index')

# Mostrar los DataFrames
print("DataFrame de Pokémon:")
print(pokemon_df)

print("\nDataFrame de Movimientos:")
print(moves_df)

DataFrame de Pokémon:
            id       name     type1   type2  \
lucario    448    lucario  fighting   steel   
pikachu     25    pikachu  electric    None   
gengar      94     gengar     ghost  poison   
charizard    6  charizard      fire  flying   
blastoise    9  blastoise     water    None   
venusaur     3   venusaur     grass  poison   
dragonite  149  dragonite    dragon  flying   
mew        151        mew   psychic    None   
mewtwo     150     mewtwo   psychic    None   
garchomp   445   garchomp    dragon  ground   

                                                       moves  
lucario    [{'name': 'mega-punch', 'url': 'https://pokeap...  
pikachu    [{'name': 'mega-punch', 'url': 'https://pokeap...  
gengar     [{'name': 'mega-punch', 'url': 'https://pokeap...  
charizard  [{'name': 'mega-punch', 'url': 'https://pokeap...  
blastoise  [{'name': 'mega-punch', 'url': 'https://pokeap...  
venusaur   [{'name': 'swords-dance', 'url': 'https://poke...  
dragonite  [{'name'

# Union de Informacion

1. Obten el nombre y caracteristicas de todos lo movimientos que aparecen en por lo menos 3 pokemons de los de la lista, y guarda los datos de movimiento en un solo `DataFrame`

In [30]:
# Obtener la lista de movimientos que aparecen en al menos 3 Pokémon
movimientos_comunes = set()

for moves_lista in pokemon_df['moves']:
    for move in moves_lista:
        movimientos_comunes.add(move['name'])

movimientos_frecuentes = [move for move in movimientos_comunes if sum(pokemon_df['moves'].apply(lambda x: move in [m['name'] for m in x])) >= 3]

# Filtrar el DataFrame de movimientos
movimientos_frecuentes_df = moves_df[moves_df['name'].isin(movimientos_frecuentes)]

# Mostrar el DataFrame de movimientos frecuentes
print("DataFrame de Movimientos Frecuentes:")
print(movimientos_frecuentes_df)

DataFrame de Movimientos Frecuentes:
                          name  accuracy      type  power  pp
mega-punch          mega-punch      85.0    normal   80.0  20
fire-punch          fire-punch     100.0      fire   75.0  15
ice-punch            ice-punch     100.0       ice   75.0  15
thunder-punch    thunder-punch     100.0  electric   75.0  15
mega-kick            mega-kick      75.0    normal  120.0   5
...                        ...       ...       ...    ...  ..
hex                        hex     100.0     ghost   65.0  10
confide                confide       NaN    normal    NaN  20
power-up-punch  power-up-punch     100.0  fighting   40.0  20
laser-focus        laser-focus       NaN    normal    NaN  30
tera-blast          tera-blast     100.0    normal   80.0  10

[99 rows x 5 columns]


1.1 Salva los datos en un archivo csv

In [31]:
movimientos_frecuentes_df.to_csv('pokemon_moves_3.csv',index=False)

2. Obten el nombre y caracteristicas todos los moviemientos que no se repiten en ningun otro pokemon, osea son movimientos unicos para un solo pokemon

In [32]:
movimientos_unicos = [move for move in movimientos_comunes if sum(pokemon_df['moves'].apply(lambda x: move in [m['name'] for m in x])) == 1]

# Filtrar el DataFrame de movimientos
movimientos_unicos_df = moves_df[moves_df['name'].isin(movimientos_unicos)]

# Mostrar el DataFrame de movimientos frecuentes
print("DataFrame de Movimientos Frecuentes:")
print(movimientos_unicos_df)

DataFrame de Movimientos Frecuentes:
                      name  accuracy    type  power  pp
lick                  lick     100.0   ghost   30.0  30
smog                  smog      70.0  poison   30.0  20
poison-gas      poison-gas      90.0  poison    NaN  40
destiny-bond  destiny-bond       NaN   ghost    NaN   5
perish-song    perish-song       NaN  normal    NaN   5
mean-look        mean-look       NaN  normal    NaN   5
shadow-punch  shadow-punch       NaN   ghost   60.0  20


2.2 Salvalos a un csv

In [33]:
movimientos_unicos_df.to_csv('unrepeated_moves.csv',index=False)

# Manejo de tipos

Para cada tipo de pokemon (fire, water, grass, etc) que aparecen en los datos. Obten una serie de dataframes que contengan a los pokemones que son de cada tipo. Y colocalos en un diccionarion donde la llave (`key`) es el tipo

In [43]:
df_tipos = {}

# Obtener todos los tipos de Pokémon presentes en el DataFrame de Pokémon
tipos_pokemon = set(pokemon_df['type1'].dropna().unique())
tipos_pokemon.update(pokemon_df['type2'].dropna().unique())

# Crear un DataFrame para cada tipo y almacenarlo en el diccionario df_tipos
for tipo in tipos_pokemon:
    # Filtrar Pokémon que son de un tipo específico en type1 o type2
    tipo_df = pokemon_df[(pokemon_df['type1'] == tipo) | (pokemon_df['type2'] == tipo)]
    
    # Almacenar el DataFrame en el diccionario
    df_tipos[tipo] = tipo_df

# Mostrar el diccionario de DataFrames por tipo
for tipo, df in df_tipos.items():
    print(f"Tipo: {tipo}")
    print(df)
    print("\n")

Tipo: dragon
            id       name   type1   type2  \
dragonite  149  dragonite  dragon  flying   
garchomp   445   garchomp  dragon  ground   

                                                       moves  
dragonite  [{'name': 'mega-punch', 'url': 'https://pokeap...  
garchomp   [{'name': 'swords-dance', 'url': 'https://poke...  


Tipo: ghost
        id    name  type1   type2  \
gengar  94  gengar  ghost  poison   

                                                    moves  
gengar  [{'name': 'mega-punch', 'url': 'https://pokeap...  


Tipo: poison
          id      name  type1   type2  \
gengar    94    gengar  ghost  poison   
venusaur   3  venusaur  grass  poison   

                                                      moves  
gengar    [{'name': 'mega-punch', 'url': 'https://pokeap...  
venusaur  [{'name': 'swords-dance', 'url': 'https://poke...  


Tipo: water
           id       name  type1 type2  \
blastoise   9  blastoise  water  None   

                               

Despues guara ese diccionario en un json llamado `tipos.json`

In [44]:
# Convertir DataFrames a lista de diccionarios
df_tipos_serializable = {tipo: df.to_dict(orient='records') for tipo, df in df_tipos.items()}

# Guardar el diccionario serializable en un archivo JSON
with open('pokemon_types.json', 'w') as json_file:
    json.dump(df_tipos_serializable, json_file)

# Seleccion de Pokemones

Ahora crea un DataFrame que para cada pokemon unicamente contenga los movimientos que son de su tipo (osea si es tipo fuego y voldaor solo usa movimientos de tipo volador y fuego). Y crea una tabla donde tengan los siguientes datos en formato `tidydata`:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  
4. `name` (del movimiento)
5. `accuracy`
6. `type`
7. `power`
8. `pp`

In [48]:
# Crear un DataFrame tidy con la información requerida
tidy_data_list = []

# Iterar sobre cada Pokémon y sus movimientos
for pokemon, info in pokemon_data_dict.items():
    pokemon_id = info['id']
    pokemon_name = info['name']
    
    # Obtener tipos directamente de las columnas type1 y type2
    type1 = pokemon_df.loc[pokemon, 'type1']
    type2 = pokemon_df.loc[pokemon, 'type2']

    types = [type1]
    if not pd.isnull(type2):
        types.append(type2)

    # Iterar sobre los movimientos del Pokémon
    for move_info in info['moves']:
        move_name = move_info['name']
        
        # Obtener información del movimiento desde el DataFrame
        move_data = movimientos_por_tipo[types[0]].loc[movimientos_por_tipo[types[0]]['name'] == move_name]
        
        # Verificar si move_data tiene algún valor
        if not move_data.empty:
            # Añadir información al DataFrame tidy
            tidy_data_list.append({
                'pokemon_id': pokemon_id,
                'pokemon_name': pokemon_name,
                'type': types,
                'name': move_name,
                'accuracy': move_data['accuracy'].values[0],
                'move_type': move_data['type'].values[0],
                'power': move_data['power'].values[0],
                'pp': move_data['pp'].values[0]
            })

# Crear el DataFrame tidy
tidy_data_df = pd.DataFrame(tidy_data_list)

# Mostrar el DataFrame tidy
print("DataFrame Tidy:")
print(tidy_data_df)


DataFrame Tidy:
     pokemon_id pokemon_name               type           name  accuracy  \
0           448      lucario  [fighting, steel]     mega-punch      85.0   
1           448      lucario  [fighting, steel]      ice-punch     100.0   
2           448      lucario  [fighting, steel]  thunder-punch     100.0   
3           448      lucario  [fighting, steel]      mega-kick      75.0   
4           448      lucario  [fighting, steel]       headbutt     100.0   
..          ...          ...                ...            ...       ...   
648         445     garchomp   [dragon, ground]      captivate     100.0   
649         445     garchomp   [dragon, ground]          round     100.0   
650         445     garchomp   [dragon, ground]        confide       NaN   
651         445     garchomp   [dragon, ground]    laser-focus       NaN   
652         445     garchomp   [dragon, ground]     tera-blast     100.0   

    move_type  power  pp  
0      normal   80.0  20  
1         ice   7

Finalmente guarda los datos en `.parquet`

In [ ]:
# Guardar el DataFrame tidy en formato Parquet
tidy_data_df.to_parquet('tabla_pokemon.parquet')